In [ ]:
pip install scikit-learn

In [ ]:
from __future__ import print_function
import time

#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
import numpy as np
import pickle
import pandas as pd
import sklearn as sk
from sklearn.svm import LinearSVC
from sklearn import metrics

pickle_protocol = min(pickle.HIGHEST_PROTOCOL, 4)

In [ ]:
with open('//drive/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Closures/Closure - Button/saved_model/3th ROUND BUTTON- 5.000 images - 2.500 keep/Closure_button_z_w_classified.pkl_pruned_dlatents_results.pkl', 'rb') as handle:
        results = pickle.load(handle)      

In [ ]:
results

In [ ]:
z = results['latents']
w = results['dlatents']
label = results['label']

In [ ]:
print('1', list(label).count(1))
print('0', list(label).count(0))

## SVM W

In [ ]:
clf_w = LinearSVC(class_weight = 'balanced')#, max_iter = 10000)
clf_w.fit(w, label)
pred_w = clf_w.predict(w)

print('svm_output = 1', list(pred_w).count(1))
print('svm_output = 0', list(pred_w).count(0))
print('Accuracy =', metrics.accuracy_score(label, pred_w))

In [ ]:
from sklearn import metrics

metrics.confusion_matrix(label, pred_w)

#print precision value of model
precision = metrics.precision_score(label, pred_w)
print(precision)

#print recall value of model
recall = metrics.recall_score(label, pred_w)
print(recall)

#print F-score
print('F-score: ', (precision * recall * 2) / (precision + recall))

In [ ]:
coef_w = clf_w.coef_
coef_w

## SVM Z

In [ ]:
clf_z = LinearSVC(class_weight = 'balanced', max_iter = 10000)
clf_z.fit(z, label)
pred_z = clf_z.predict(z)

print('svm_output = 1', list(pred_z).count(1))
print('svm_output = 0', list(pred_z).count(0))
print('Accuracy =', metrics.accuracy_score(label, pred_z))

In [ ]:
coef_z = clf_z.coef_
coef_z

## Save latent codes and coefficients of both Z and W

## W

In [ ]:
dlatent_image = w[1274]

save_dlatent_image = '//drive/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Closures/Closure - Button/saved_model/3th ROUND BUTTON- 5.000 images - 2.500 keep/dlatent_image.pkl'

with open(save_dlatent_image, 'wb') as handle:
    pickle.dump(dlatent_image, handle, protocol=pickle_protocol)

In [ ]:
save_svm_coef_dlatent = '//drive/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Closures/Closure - Button/saved_model/3th ROUND BUTTON- 5.000 images - 2.500 keep/svm_coef_dlatent.pkl'

with open(save_svm_coef_dlatent, 'wb') as handle:
    pickle.dump(coef_w, handle, protocol=pickle_protocol)

## Z

In [ ]:
latent_image = z[1274]

save_latent_image = '//drive/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Closures/Closure - Button/saved_model/3th ROUND BUTTON- 5.000 images - 2.500 keep/latent_image.pkl'

with open(save_latent_image, 'wb') as handle:
    pickle.dump(latent_image, handle, protocol=pickle_protocol)

In [ ]:
save_svm_coef_latent = '//drive/Algemeen/Interns/Mirle Willems/CLASSIFIER DATA/Closures/Closure - Button/saved_model/3th ROUND BUTTON- 5.000 images - 2.500 keep/svm_coef_latent.pkl'

with open(save_svm_coef_latent, 'wb') as handle:
    pickle.dump(coef_z, handle, protocol=pickle_protocol)

# Conditional Entropy

In [ ]:
def prob_normalize(p):
    p = np.asarray(p).astype(np.float32)
    assert len(p.shape) == 2
    return p / np.sum(p)

def mutual_information(p):
    p = prob_normalize(p)
    px = np.sum(p, axis=1)
    py = np.sum(p, axis=0)
    result = 0.0
    for x in range(p.shape[0]):
        p_x = px[x]
        for y in range(p.shape[1]):
            p_xy = p[x][y]
            p_y = py[y]
            if p_xy > 0.0:
                result += p_xy * np.log2(p_xy / (p_x * p_y)) # get bits as output
    return result

def entropy(p):
    p = prob_normalize(p)
    result = 0.0
    for x in range(p.shape[0]):
        for y in range(p.shape[1]):
            p_xy = p[x][y]
            if p_xy > 0.0:
                result -= p_xy * np.log2(p_xy)
    return result

def conditional_entropy(p):
    # H(Y|X) where X corresponds to axis 0, Y to axis 1
    # i.e., How many bits of additional information are needed to where we are on axis 1 if we know where we are on axis 0?
    p = prob_normalize(p)
    y = np.sum(p, axis=0, keepdims=True) # marginalize to calculate H(Y)
    return max(0.0, entropy(y) - mutual_information(p)) # can slip just below 0 due to FP inaccuracies, clean those up.

# Calculate conditional entropy.
p = [[np.mean([case == (row, col) for case in zip(pred_w, label)]) for col in (0, 1)] for row in (0, 1)]

conditional_cross = conditional_entropy(p)

print(conditional_cross)

## Visualisation T-sne

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(w)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
#svm_inputs['tsne-2d-one'] = tsne_results[:,0]
#svm_inputs['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))

sns.scatterplot(
    x=tsne_results[:,0], y=tsne_results[:,1],
    hue=label,
    #palette=sns.color_palette("hls", 10),
    data=w,
    legend="full",
    alpha=0.3
)